# **Vanilla BERT Text Encoding**

In this colab, I will be getting the text encodings for the queries using a Vanilla BERT model.  These will be saved for future reference.

## **Setup**

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 30.8 MB/s 
     |████████████████████████████████| 6.8 MB 32.9 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 42.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install --upgrade python-terrier

     |████████████████████████████████| 97 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 35.2 MB/s 
     |████████████████████████████████| 69 kB 6.0 MB/s 
     |████████████████████████████████| 255 kB 4.2 MB/s 
     |████████████████████████████████| 41 kB 279 kB/s 
     |████████████████████████████████| 1.8 MB 31.0 MB/s 
     |████████████████████████████████| 294 kB 41.6 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 33.8 MB/s 
     |████████████████████████████████| 126 kB 36.9 MB/s 
     |████████████████████████████████| 291 kB 33.4 MB/s 
     |████████████████████████████████| 59 kB 3.7 MB/s 
     |████████████████████████████████| 45 kB 2.1 MB/s 


In [ ]:
import pyterrier as pt

if not pt.started():
  pt.init()

In [2]:
import pandas as pd
import pickle

In [ ]:
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = cord19.get_topics('title')

In [3]:
!wget https://github.com/DavidONeill75101/level-4-project/blob/master/Datasets/CORD-19_Datasets/round5_docs.csv?raw=true
round5_docs = pd.read_csv('/content/round5_docs.csv?raw=true').drop(columns=['Unnamed: 0'])

--2022-03-15 14:09:09--  https://github.com/DavidONeill75101/level-4-project/blob/master/Datasets/CORD-19_Datasets/round5_docs.csv?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/DavidONeill75101/level-4-project/raw/master/Datasets/CORD-19_Datasets/round5_docs.csv [following]
--2022-03-15 14:09:09--  https://github.com/DavidONeill75101/level-4-project/raw/master/Datasets/CORD-19_Datasets/round5_docs.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/DavidONeill75101/level-4-project/master/Datasets/CORD-19_Datasets/round5_docs.csv [following]
--2022-03-15 14:09:10--  https://media.githubusercontent.com/media/DavidONeill75101/level-4-project/master/Datasets/CORD-19_Datasets/round5_docs.csv
Resolving media.githubusercontent.com (me

## **Prepare for Encoding**

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",output_hidden_states=True)

model = model.to(device)

The following function will encode the query concatenated to the document with a '\n' character in between.

In [ ]:
def text_to_embed(text):
  # Tokenize it with appropriate padding and truncation
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=500)

  # Move the IDs of the tokens over to the GPU
  input_ids = inputs['input_ids'].to(device)

  # Run the model on the data
  outputs = model(input_ids=input_ids)

  # Extract the embeddings
  with torch.no_grad():
    # Get the final layer of the neural network, and average the embedding for all the tokens
    # Some researchers use the vector just for the first or final token of the sentence
    # instead of an average. I don't think there is a definitive best approach.
    # You could stick to the mean for now.
    embed = outputs.hidden_states[-1].squeeze().mean(axis=0)

    # Return the embedding to the CPU and convert to a numpy array
    embed = embed.cpu().numpy()

  return embed

## **Get Query Embeddings**

In [ ]:
query_embeddings = {}
for i in range(1, 51):
  query_embeddings[str(i)] = text_to_embed(topics.iloc[i-1]['query'])
